<a href="https://colab.research.google.com/github/DousMoon/ml0804/blob/master/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from urllib.request import urlretrieve
url = "https://github.com/Elwing-Chou/ml0804/raw/master/train.csv"
urlretrieve(url, "train.csv")
url = "https://github.com/Elwing-Chou/ml0804/raw/master/test.csv"
urlretrieve(url, "test.csv")

In [ ]:
import pandas as pd
train_df = pd.read_csv("train.csv", encoding="utf-8")
train_df

In [ ]:
test_df = pd.read_csv("test.csv", encoding="utf-8")
test_df

In [ ]:
# concat: axis!
import pandas as pd
datas = pd.concat([train_df,test_df]).reset_index(drop=True)
datas = datas.drop(["PassengerId","Survived"],axis=1)
datas

In [ ]:
s = datas.isna().sum() # isna沒有值=True
s[s>0].sort_values(ascending=False) # 大到小:s[s>0].sort_values(ascending = False)

In [ ]:
# unique/value_counts
# 類別請補最常出現
most = datas["Embarked"].value_counts().idxmax()
datas["Embarked"] = datas["Embarked"].fillna(most)

In [ ]:
# 數值補中位數
med = datas.median().drop(["Pclass"])
datas = datas.fillna(med)
med

In [ ]:
# apply:
def cabin(s):
  if not pd.isna(s):
    return s[0]
  else:
    return None
datas["Cabin"] = datas["Cabin"].apply(cabin)


In [ ]:
v = datas["Ticket"].value_counts()
def ticket(s):
    global v
    return v[s]
datas["Ticket"] = datas["Ticket"].apply(ticket)

In [ ]:
def name(s):
  return s.split(".")[0].split(",")[-1].strip()
n = datas["Name"].apply(name).value_counts()
whitelist = n[:4]

In [ ]:
def name2(s):
    mid = s.split(".")[0].split(",")[-1].strip()
    if mid in whitelist:
        return mid
    else:
        return None
datas["Name"] = datas["Name"].apply(name2)
datas

In [ ]:
# 指定做:colimns["Cabin"]
datas = pd.get_dummies(datas)
datas

In [ ]:
# 新增自訂欄位
datas["Family"] = datas["SibSp"]+datas["Parch"]
# datas

In [ ]:
# .iloc取列 [第一列, 第二列, 第三列...]
# 0~len-1
x_train = datas.iloc[:len(train_df)]
x_predict = datas.iloc[len(train_df):]
y_train = train_df["Survived"]

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
clf = RandomForestClassifier(random_state=0)
params = {
  "n_estimators":range(20, 100),
  "max_depth":range(5, 10)
}
grid = GridSearchCV(clf, params, cv=10, n_jobs=-1)
grid.fit(x_train, y_train)
print(grid.best_params_)
print(grid.best_score_)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import numpy as np
clf = RandomForestClassifier(n_estimators=37, max_depth=6)
scores = cross_val_score(clf, x_train, y_train, cv=10, n_jobs=-1)
print("[10]:",scores)
print("average:",np.average(scores))

In [ ]:
clf = RandomForestClassifier(n_estimators=37,max_depth=6)
clf.fit(x_train, y_train)
pre = clf.predict(x_predict)
ans =  pd.DataFrame({
    "PassengerId":test_df["PassengerId"],
    "Survived":pre
})
ans.to_csv("rf.csv",encoding="utf-8",index=False)
ans